In [24]:
import json
import numpy as np
import nltk
from nltk.stem.snowball import SnowballStemmer
import torch
import torch.nn as nn
import pickle

from src.stopword_remover import remove_stopwords

from src.data_loader import load_training_data

training, output, all_words, tags = load_training_data()

# Definir modelo
class ChatbotModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate):
        super(ChatbotModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self, x):
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = nn.functional.softmax(x, dim=1)
        return x

model = ChatbotModel(len(training[0]), 128, len(output[0]), 0.5)

# Entrenar modelo
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

for epoch in range(100):
    inputs = torch.from_numpy(training).float()
    outputs = model(inputs)
    
    loss = criterion(outputs, output)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Guardar modelo en un archivo pkl
with open('models/chatbot_model.pkl', 'wb') as file:
    pickle.dump(model, file)


In [26]:
import json
import numpy as np
import nltk
from nltk.stem.snowball import SnowballStemmer
import torch
import torch.nn as nn
import pickle

from src.stopword_remover import remove_stopwords

from src.chatbot import Chatbot

# Cargar modelo desde archivo pkl
chatbot = Chatbot('models/chatbot_model.pkl')

# Obtener respuesta del chatbot
while True:
    message = input('You: ')
    if message.lower() == 'exit':
        break
    response = chatbot.get_response(message)
    print('Chatbot:', response)


TypeError: expected string or bytes-like object

In [21]:
import json
import numpy as np
import nltk
from nltk.stem.snowball import SnowballStemmer
import torch
import torch.nn as nn
import pickle

from src.stopword_remover import remove_stopwords
from src.data_loader import load_training_data

import random

training, output, all_words, tags = load_training_data()

class ChatbotModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate):
        super(ChatbotModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = nn.functional.softmax(x, dim=1)
        return x

class Chatbot:
    def __init__(self, model_path):
        with open(model_path, 'rb') as file:
            self.model = pickle.load(file)
        
        with open('data/intents.json', encoding='iso-8859-1') as file:
            self.intents = json.load(file)
        
        self.all_words = all_words
        self.tags = tags
    
    def clean_up_sentence(self, sentence):
        # Tokenización y eliminación de stopwords
        sentence_words = nltk.word_tokenize(sentence)
        sentence_words = remove_stopwords(sentence_words)
        # Stemming
        stemmer = SnowballStemmer('spanish')
        sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
        return sentence_words

    def bag_of_words(self, sentence):
        sentence_words = self.clean_up_sentence(sentence)
        bag = np.zeros(len(self.all_words), dtype=np.float32)
        for idx, word in enumerate(self.all_words):
            if word in sentence_words:
                bag[idx] = 1
        return bag

    def predict_tag(self, sentence):
        # Obtener predicción de etiqueta
        bow = self.bag_of_words(sentence)
        x = torch.from_numpy(bow).unsqueeze(0)
        output = self.model(x)
        _, predicted = torch.max(output, dim=1)
        tag = self.tags[predicted.item()]
        return tag

    def preprocess(input_text, words):
        # Tokenize the input text
        tokens = nltk.word_tokenize(input_text.lower())

        # Stem the tokens
        stemmer = SnowballStemmer('spanish')
        stemmed_tokens = [stemmer.stem(token) for token in tokens]

        # Create a bag of words vector
        input_bag = torch.zeros(len(words))
        for word in stemmed_tokens:
            if word in words:
                input_bag[words.index(word)] = 1

        return input_bag
    def get_response(self, sentence):
        # Preprocess the input text
    input_bag = preprocess(input_text, words)
    #input_bag = preprocess_input(input_text)
    #print(input_bag)

    # Convert input to a PyTorch tensor
    #input_tensor = torch.tensor(input_bag).float().unsqueeze(0)
    #input_tensor = torch.tensor(input_bag, dtype=torch.float).clone().detach().unsqueeze(0)
    input_tensor = input_bag.clone().detach().unsqueeze(0)

    # Use the model to predict the output
    output = model(input_tensor)

    # Convert the output to a numpy array
    output_np = output.detach().numpy()

    # Find the tag with the highest probability
    tag_index = np.argmax(output_np)

    if output_np[0][tag_index] < 0.5:
        return "Lo siento, no te he entendido. ¿Podrías reformular la pregunta?"

    tag = tags[tag_index]

    # Choose a random response from the tag's list of responses
    responses = []
    for intent in data['intents']:
        if intent['tag'] == tag:
            responses = intent['responses']

    return random.choice(responses)
        # Obtener respuesta de acuerdo a la etiqueta
        tag = self.predict_tag(sentence)
        for intent in self.intents['intents']:
            if intent['tag'] == tag:
                responses = intent['responses']
        return np.random.choice(responses)


IndentationError: expected an indented block after function definition on line 83 (4035665283.py, line 85)

In [22]:
import pickle
import numpy as np
from nltk.stem import SnowballStemmer

# Load the model from the file
with open('models/chatbot_model.pkl', 'rb') as file:
    model = pickle.load(file)

# Load the words and tags from the data file
with open('data/intents.json', encoding='iso-8859-1') as file:
    data = json.load(file)

stemmer = SnowballStemmer('spanish')
words = []
tags = []
patterns = []

for intent in data['intents']:
    tag = intent['tag']
    tags.append(tag)
    
    for pattern in intent['patterns']:
        # Tokenize and stem the words in the pattern
        tokens = nltk.word_tokenize(pattern.lower())
        stemmed_tokens = [stemmer.stem(token) for token in tokens]
        words.extend(stemmed_tokens)
        patterns.append((stemmed_tokens, tag))

words = sorted(list(set(words)))
tags = sorted(list(set(tags)))

In [23]:
model_path = 'models/chatbot_model.pkl'
chatbot = Chatbot(model_path)
while True:
    user_input = input("You: ")
    response = chatbot.get_response(user_input)
    print("Chatbot:", response)

TypeError: expected string or bytes-like object

In [11]:
def preprocess(input_text, words):
    # Tokenize the input text
    tokens = nltk.word_tokenize(input_text.lower())

    # Stem the tokens
    stemmer = SnowballStemmer('spanish')
    stemmed_tokens = [stemmer.stem(token) for token in tokens]

    # Create a bag of words vector
    input_bag = torch.zeros(len(words))
    for word in stemmed_tokens:
        if word in words:
            input_bag[words.index(word)] = 1

    return input_bag

In [12]:
import random

def generate_response(input_text):
    # Preprocess the input text
    input_bag = preprocess(input_text, words)
    #input_bag = preprocess_input(input_text)
    #print(input_bag)

    # Convert input to a PyTorch tensor
    #input_tensor = torch.tensor(input_bag).float().unsqueeze(0)
    #input_tensor = torch.tensor(input_bag, dtype=torch.float).clone().detach().unsqueeze(0)
    input_tensor = input_bag.clone().detach().unsqueeze(0)

    # Use the model to predict the output
    output = model(input_tensor)

    # Convert the output to a numpy array
    output_np = output.detach().numpy()

    # Find the tag with the highest probability
    tag_index = np.argmax(output_np)

    if output_np[0][tag_index] < 0.5:
        return "Lo siento, no te he entendido. ¿Podrías reformular la pregunta?"

    tag = tags[tag_index]

    # Choose a random response from the tag's list of responses
    responses = []
    for intent in data['intents']:
        if intent['tag'] == tag:
            responses = intent['responses']

    return random.choice(responses)

In [13]:
generate_response('Hola')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x108 and 31x128)

In [27]:
import json
import numpy as np
import nltk
from nltk.stem.snowball import SnowballStemmer
import torch
import torch.nn as nn
import pickle

from src.stopword_remover import remove_stopwords

from src.data_loader import load_training_data

training, output, all_words, tags = load_training_data()

# Definir modelo
class ChatbotModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate):
        super(ChatbotModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self, x):
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = nn.functional.softmax(x, dim=1)
        return x

model = ChatbotModel(len(training[0]), 128, len(output[0]), 0.5)

# Entrenar modelo
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

for epoch in range(100):
    inputs = torch.from_numpy(training).float()
    outputs = model(inputs)
    
    loss = criterion(outputs, output)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Guardar modelo en un archivo pkl
with open('models/chatbot_model.pkl', 'wb') as file:
    pickle.dump(model, file)


In [38]:
import json
import numpy as np
import nltk
from nltk.stem.snowball import SnowballStemmer
import torch
import torch.nn as nn
import pickle

from src.stopword_remover import remove_stopwords

from src.data_loader import load_training_data

#training, output, all_words, tags = load_training_data()
training, output, words, tags = load_training_data()

def preprocess(input_text, words):
    # Tokenize the input text
    tokens = nltk.word_tokenize(input_text.lower())

    # Stem the tokens
    stemmer = SnowballStemmer('spanish')
    stemmed_tokens = [stemmer.stem(token) for token in tokens]

    print(stemmed_tokens)

    # Create a bag of words vector
    input_bag = torch.zeros(len(words))
    for word in stemmed_tokens:
        if word in words:
            input_bag[words.index(word)] = 1

    return input_bag
def generate_response(input_text):
    # Preprocess the input text
    input_bag = preprocess(input_text, words)
    #input_bag = preprocess_input(input_text)
    print(input_bag.shape)

    # Convert input to a PyTorch tensor
    input_tensor = input_bag.clone().detach().unsqueeze(0)

    # Use the model to predict the output
    output = model(input_tensor)

    # Convert the output to a numpy array
    output_np = output.detach().numpy()

    # Find the tag with the highest probability
    tag_index = np.argmax(output_np)

    if output_np[0][tag_index] < 0.5:
        return "Lo siento, no te he entendido. ¿Podrías reformular la pregunta?"

    tag = tags[tag_index]

    # Choose a random response from the tag's list of responses
    responses = []
    for intent in data['intents']:
        if intent['tag'] == tag:
            responses = intent['responses']

    return random.choice(responses)
generate_response('Hola')

['hol']
torch.Size([31])


'¡Que tengas un buen día!'

In [39]:
import pickle
import numpy as np
from nltk.stem import SnowballStemmer

# Load the model from the file
with open('models/chatbot_model.pkl', 'rb') as file:
    model = pickle.load(file)

# Load the words and tags from the data file
with open('data/intents.json', encoding='iso-8859-1') as file:
    data = json.load(file)

stemmer = SnowballStemmer('spanish')
words = []
tags = []
patterns = []

for intent in data['intents']:
    tag = intent['tag']
    tags.append(tag)
    
    for pattern in intent['patterns']:
        # Tokenize and stem the words in the pattern
        tokens = nltk.word_tokenize(pattern.lower())
        stemmed_tokens = [stemmer.stem(token) for token in tokens]
        words.extend(stemmed_tokens)
        patterns.append((stemmed_tokens, tag))

words = sorted(list(set(words)))
tags = sorted(list(set(tags)))

In [29]:
def preprocess(input_text, words):
    # Tokenize the input text
    tokens = nltk.word_tokenize(input_text.lower())

    # Stem the tokens
    stemmer = SnowballStemmer('spanish')
    stemmed_tokens = [stemmer.stem(token) for token in tokens]

    # Create a bag of words vector
    input_bag = torch.zeros(len(words))
    for word in stemmed_tokens:
        if word in words:
            input_bag[words.index(word)] = 1

    return input_bag

In [30]:
def generate_response(input_text):
    # Preprocess the input text
    input_bag = preprocess(input_text, words)
    #input_bag = preprocess_input(input_text)
    #print(input_bag)

    # Convert input to a PyTorch tensor
    #input_tensor = torch.tensor(input_bag).float().unsqueeze(0)
    #input_tensor = torch.tensor(input_bag, dtype=torch.float).clone().detach().unsqueeze(0)
    input_tensor = input_bag.clone().detach().unsqueeze(0)

    # Use the model to predict the output
    output = model(input_tensor)

    # Convert the output to a numpy array
    output_np = output.detach().numpy()

    # Find the tag with the highest probability
    tag_index = np.argmax(output_np)

    if output_np[0][tag_index] < 0.5:
        return "Lo siento, no te he entendido. ¿Podrías reformular la pregunta?"

    tag = tags[tag_index]

    # Choose a random response from the tag's list of responses
    responses = []
    for intent in data['intents']:
        if intent['tag'] == tag:
            responses = intent['responses']

    return random.choice(responses)

In [31]:
generate_response('Hola')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x108 and 31x128)

In [1]:
import json
import numpy as np
import nltk
from nltk.stem.snowball import SnowballStemmer
import torch
import torch.nn as nn
import pickle

from src.stopword_remover import remove_stopwords

from src.data_loader import load_training_data

training, output, all_words, tags, data = load_training_data()

# Definir modelo
class ChatbotModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate):
        super(ChatbotModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self, x):
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = nn.functional.softmax(x, dim=1)
        return x

#model = ChatbotModel(len(training[0]), 128, len(output[0]), 0.5)
model = ChatbotModel(len(training[0]), 128, len(output[0]), 0.5)

# Entrenar modelo
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

for epoch in range(100):
    inputs = torch.from_numpy(training).float()
    outputs = model(inputs)
    
    loss = criterion(outputs, output)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Guardar modelo en un archivo pkl
with open('models/chatbot_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [2]:
all_words

['!',
 ',',
 '.',
 '?',
 'a',
 'adios',
 'agradezc',
 'avanz',
 'ayud',
 'ayudart',
 'bienven',
 'buen',
 'carrer',
 'com',
 'con',
 'conmig',
 'consej',
 'consult',
 'curriculum',
 'de',
 'deb',
 'decision',
 'deprim',
 'dia',
 'dias',
 'dorm',
 'el',
 'en',
 'encontr',
 'entiend',
 'entrev',
 'es',
 'escuch',
 'estas',
 'esto',
 'estoy',
 'estres',
 'graci',
 'gust',
 'ha',
 'habl',
 'hac',
 'hast',
 'hol',
 'interpersonal',
 'jef',
 'la',
 'las',
 'line',
 'lueg',
 'mal',
 'me',
 'mejor',
 'mi',
 'mis',
 'much',
 'muy',
 'nadi',
 'necesit',
 'no',
 'noch',
 'nos',
 'par',
 'personal',
 'piens',
 'por',
 'prepar',
 'problem',
 'pront',
 'proxim',
 'pued',
 'que',
 'qui',
 'recomiend',
 'relacion',
 'ruptur',
 'se',
 'sent',
 'sid',
 'sient',
 'situacion',
 'super',
 'tal',
 'tard',
 'te',
 'ten',
 'teng',
 'tom',
 'trabaj',
 'trat',
 'trist',
 'tu',
 'un',
 'una',
 'util',
 'valor',
 'vem',
 'vid',
 '¡cha',
 '¡hast',
 '¡hol',
 '¡qu',
 '¿com',
 '¿cual',
 '¿en',
 '¿necesit',
 '¿pued',


In [3]:
tags

['agradecimiento',
 'consejo',
 'despedida',
 'problema_personal',
 'problema_profesional',
 'saludo']

In [6]:
import json
import numpy as np
import nltk
from nltk.stem.snowball import SnowballStemmer
import torch
import torch.nn as nn
import pickle

import random

from src.stopword_remover import remove_stopwords

from src.data_loader import load_training_data

#training, output, all_words, tags = load_training_data()
training, output, words, tags, data = load_training_data()

def preprocess(input_text, words):
    # Tokenize the input text
    tokens = nltk.word_tokenize(input_text.lower())

    # Stem the tokens
    stemmer = SnowballStemmer('spanish')
    stemmed_tokens = [stemmer.stem(token) for token in tokens]

    print(stemmed_tokens)

    # Create a bag of words vector
    input_bag = torch.zeros(len(words))
    for word in stemmed_tokens:
        if word in words:
            input_bag[words.index(word)] = 1

    return input_bag
def generate_response(input_text):
    # Preprocess the input text
    input_bag = preprocess(input_text, words)
    #input_bag = preprocess_input(input_text)
    print(input_bag.shape)

    # Convert input to a PyTorch tensor
    input_tensor = input_bag.clone().detach().unsqueeze(0)

    # Use the model to predict the output
    output = model(input_tensor)

    # Convert the output to a numpy array
    output_np = output.detach().numpy()

    # Find the tag with the highest probability
    tag_index = np.argmax(output_np)

    if output_np[0][tag_index] < 0.5:
        return "Lo siento, no te he entendido. ¿Podrías reformular la pregunta?"

    tag = tags[tag_index]

    # Choose a random response from the tag's list of responses
    responses = []
    for intent in data['intents']:
        if intent['tag'] == tag:
            responses = intent['responses']

    return random.choice(responses)
generate_response('hola')

['hol']
torch.Size([108])


'Hola, bienvenido a mi consulta en línea. ¿En qué puedo ayudarte?'

In [11]:
training.shape

(48, 31)

In [12]:
output.shape

torch.Size([48, 6])

In [16]:
words

[' ',
 '!',
 ',',
 '.',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'x',
 'y',
 'z',
 '¡',
 '¿']

In [15]:
data

['agradecimiento',
 'consejo',
 'despedida',
 'problema_personal',
 'problema_profesional',
 'saludo']